In [71]:
%pip install keras-flops

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Alex\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [72]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [73]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [74]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [75]:
# Преобразуйте метки классов в one_hot формат
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train)
y_train = enc.transform(y_train).toarray()
enc.fit(y_val)
y_val = enc.transform(y_val).toarray()

In [76]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [77]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(16, 2, padding='same'),
    tf.keras.layers.MaxPooling2D((3,3), strides=4),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [78]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6264e6


In [79]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_29 (Conv2D)          (None, 32, 32, 16)        208       
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 8, 8, 16)         0         
 g2D)                                                            
                                                                 
 batch_normalization_21 (Bat  (None, 8, 8, 16)         64        
 chNormalization)                                                
                                                                 
 activation_40 (Activation)  (None, 8, 8, 16)          0         
                                                                 
 flatten_20 (Flatten)        (None, 1024)              0         
                                                                 
 dense_20 (Dense)            (None, 100)             

In [80]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [81]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=50
)

Epoch 1/50
391/391 [==============================] - 5s 11ms/step - loss: 3.4856 - accuracy: 0.2151 - val_loss: 3.1897 - val_accuracy: 0.2674
Epoch 2/50
391/391 [==============================] - 4s 11ms/step - loss: 2.7949 - accuracy: 0.3279 - val_loss: 3.0530 - val_accuracy: 0.2855
Epoch 3/50
391/391 [==============================] - 4s 11ms/step - loss: 2.5631 - accuracy: 0.3729 - val_loss: 2.9580 - val_accuracy: 0.3066
Epoch 4/50
391/391 [==============================] - 4s 11ms/step - loss: 2.4034 - accuracy: 0.4053 - val_loss: 2.9172 - val_accuracy: 0.3141
Epoch 5/50
391/391 [==============================] - 4s 11ms/step - loss: 2.2772 - accuracy: 0.4316 - val_loss: 2.8977 - val_accuracy: 0.3224
Epoch 6/50
391/391 [==============================] - 4s 11ms/step - loss: 2.1776 - accuracy: 0.4551 - val_loss: 2.9228 - val_accuracy: 0.3236
Epoch 7/50
391/391 [==============================] - 4s 11ms/step - loss: 2.0954 - accuracy: 0.4742 - val_loss: 2.9207 - val_accuracy: 0.3309

In [82]:
tf.get_logger().setLevel("ERROR")

model.load_weights("05-0.32.hdf5")
score = model.evaluate(X_val, y_val, verbose=0)
print(f"Test loss: {round(score[0], 3)}")
print(f"Test accuracy: {round(score[1], 3)}")

flops = get_flops(model)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

Test loss: 2.898
Test accuracy: 0.322
FLOPs: 0.6264e6
